## Setup PyDrake using underactuated in Google Colab

In [ ]:
import sys
import importlib
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_underactuated
  setup_underactuated()

## Import Libraries

In [ ]:
# Imports
import os
import numpy as np

from pydrake.all import MultibodyPlant, Parser

from pydrake.all import DirectTranscription, DirectCollocation, Solve

from pydrake.autodiffutils import AutoDiffXd

## Setup URDF Model Path

In [ ]:
if 'google.colab' in sys.modules:
  # Check if this cell has already been run:
  if not os.path.isdir('DrakeTests/boxTrajOpt/'):
    !git clone https://github.com/vyas-shubham/DrakeTests.git
  urdfFolderPath = 'DrakeTests/boxTrajOpt/'
else:
  urdfFolderPath = ''

# Comment/Uncomment the lines below to select a URDF
urdfpath = urdfFolderPath+'box.urdf'
assert os.path.isfile(urdfpath)

## Optimization Problem

The current problem is taken from **Betts, J. T. (2010). Practical Methods for Optimal Control and Estimation Using Nonlinear Programming.** 

**Example 6.12: Reorientation of a Rigid Body.**

The urdf contains the inertias as given in the book: Ixx = 5621, Iyy = 4547, Izz = 2364

Torque Limits = abs(u) <= 50Nm

$\phi = 150^o$

Initial State: Rotation Quaternion q(0) = [1,0,0,0]. Position Vector = [0,0,0]

Final State: Rotation Quaternion q(N) = $[cos(\phi/2),sin(\phi/2),0,0]$. Position Vector = [0,0,0]

The trajectory optimisation is setup using generalized forces as the inputs. No actuators are used.

Since this is similar to a double integrator in rotation form, the optimal solution is a bang-bang control with max/min torques being applied to each of the rotation axes. The time taken (for one of the solutions) is 28.63s. 

### Initial and Final State Arrays

In [ ]:
x0_base_position = [1, 0, 0, 0, 0, 0, 0]
x0_base_vel = [0, 0, 0, 0, 0, 0]
x0 = np.array(x0_base_position + x0_base_vel)

phi = np.deg2rad(150)
xf_base_position = [np.cos(phi/2), np.sin(phi/2), 0, 0, 0, 0, 0]
xf_base_vel = [0, 0, 0, 0, 0, 0]
xf = np.array(xf_base_position + xf_base_vel)

### Using Direct Transciption

The solver is not able to find a solution to due infeasible constraints.

In [ ]:
# # Direct Transcription Method

dt = 0.1

plant = MultibodyPlant(time_step=dt)
Parser(plant).AddModelFromFile(urdfpath)
# Remove gravity
plantGravityField = plant.gravity_field()
plantGravityField.set_gravity_vector([0,0,0])
plant.Finalize()
# plant = plant.ToAutoDiffXd()

trajOptContext = plant.CreateDefaultContext()

NDirTrans = 300


prog = DirectTranscription(plant, trajOptContext, NDirTrans,
                           input_port_index=plant.get_applied_generalized_force_input_port().get_index())

for x_index in range(x0.size):
    prog.AddConstraint(prog.initial_state()[x_index] == x0[x_index])
    prog.AddConstraint(prog.final_state()[x_index] == xf[x_index])

for u_index in range(prog.input().size):
    prog.AddConstraintToAllKnotPoints(prog.input()[u_index] <= 50)
    prog.AddConstraintToAllKnotPoints(prog.input()[u_index] >= 50)


    
result = Solve(prog)

print("Optimisation Solution Found?: {}".format(result.is_success()))

if not result.is_success():
    infeasible = result.GetInfeasibleConstraints(prog)
    print("Infeasible constraints:")
    for i in range(len(infeasible)):
        print(infeasible[i])

### Using Direct Collocation

The solver is not able to find a solution to due infeasible constraints.

In [ ]:
# Direct Collocation Method

plant = MultibodyPlant(time_step=0.0)
Parser(plant).AddModelFromFile(urdfpath)
# Remove gravity
plantGravityField = plant.gravity_field()
plantGravityField.set_gravity_vector([0,0,0])
plant.Finalize()
# plant = plant.ToAutoDiffXd()
trajOptContext = plant.CreateDefaultContext()

NDirCol = 60
h_min = 0.01
h_max = 0.5

dircol = DirectCollocation(plant,
                           trajOptContext,
                           num_time_samples=NDirCol,
                           minimum_timestep=h_min,
                           maximum_timestep=h_max,
                           input_port_index=plant.get_applied_generalized_force_input_port().get_index())

dircol.AddEqualTimeIntervalsConstraints()

for x_index in range(x0.size):
    dircol.AddConstraint(dircol.initial_state()[x_index] == x0[x_index])
    dircol.AddConstraint(dircol.final_state()[x_index] == xf[x_index])
    
for u_index in range(dircol.input().size):
    prog.AddConstraintToAllKnotPoints(prog.input()[u_index] <= 50)
    prog.AddConstraintToAllKnotPoints(prog.input()[u_index] >= -50)

result = Solve(prog)

print("Optimisation Solution Found?: {}".format(result.is_success()))

if not result.is_success():
    infeasible = result.GetInfeasibleConstraints(prog)
    print("Infeasible constraints:")
    for i in range(len(infeasible)):
        print(infeasible[i])